In [1]:
from credentials import PINECONE_API_KEY
from pinecone import Pinecone
from json import dumps

from vertexai.language_models import TextEmbeddingModel
from langchain.text_splitter import SpacyTextSplitter

pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index("infofin")

gecko = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

In [ ]:
index.delete(delete_all=True)

with open("./Memoria_BCH_2022_interactiva-10-05.md", "r") as f:
    markdown_text = f.read()
    
text_splitter = SpacyTextSplitter(
    pipeline="es_dep_news_trf",
    chunk_size=500,
    chunk_overlap=50
)

docs = text_splitter.split_text(markdown_text)
print(len(docs))



chunk_number = 0
for doc in docs:
    text_embedding_input = TextEmbeddingInput(
        text=doc,
        task_type="RETRIEVAL_DOCUMENT"
    )
    vector = gecko.get_embeddings([text_embedding_input])
    index.upsert([(str(chunk_number), vector[0].values, {"text": doc})])
    chunk_number = chunk_number + 1

In [2]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.language_models import TextEmbeddingInput

gemini = GenerativeModel("gemini-pro")

def ask_gemini(question: str, print_prompt=False) -> str:
    
    text_embedding_input = TextEmbeddingInput(
        text=question,
        task_type="RETRIEVAL_QUERY"
    )
    
    vector = gecko.get_embeddings([text_embedding_input])
    
    matches = index.query(
        vector=vector[0].values,
        top_k=10,
        include_metadata=True
    )

    # " Si la respuesta no se encuentra en el documento, indica que la información no está en el documento." \
    # " Siempre debes responder en primera persona como un empleado del banco." \
    # " Responde en un párrafo o menos." \
    prompt = \
        "Contexto: Eres experto en responder preguntas sobre el Banco de Chile." \
        " En el siguiente documento se encuentran los extractos que debes usar para responder." \
        " Siempre debes responder en primera persona como un empleado del banco." \
        f"\nDocumento: {dumps(matches.to_dict(), ensure_ascii=False)}" \
        f"\nPregunta: ¿Quién es el Gerente General de Banco de Chile?" \
        f"\nRespuesta: Eduardo Ebenperger Orrego." \
        f"\nPregunta: ¿Cuántos años tiene Banco de Chile operando en el país?" \
        f"\nRespuesta: 129 años." \
        f"\nPregunta: ¿Cuál es la misión del Banco de Chile?" \
        f"\nRespuesta: Contribuir al desarrollo del país y al progreso de las personas, mejorando su calidad de vida sin comprometer el bienestar de las futuras generaciones. Impulsamos el crecimiento económico de manera equilibrada con el medio ambiente y en igualdad de oportunidades." \
        f"\nPregunta: {question}" \
        f"\nRespuesta:"
    
    if print_prompt:
        print(prompt)
    
    gemini_response = gemini.generate_content(prompt, stream=False)
    
    try:
        return gemini_response.text
    except:
        return "Error!"

In [3]:
ask_gemini("¿Cuántas sucursales hay en Tarapacá?", print_prompt=False)

'4 sucursales.'

In [ ]:
from vertexai.language_models import TextGenerationModel

unicorn = TextGenerationModel.from_pretrained("text-unicorn@001")

def ask_unicorn(question: str, print_prompt=False) -> str:
    
    text_embedding_input = TextEmbeddingInput(
        text=question,
        task_type="RETRIEVAL_QUERY"
    )
    
    vector = gecko.get_embeddings([text_embedding_input])
    

    matches = index.query(
        vector=vector[0].values,
        top_k=10,
        include_metadata=True
    )

    # " Si la respuesta no se encuentra en el documento, indica que la información no está en el documento." \
    # " Siempre debes responder en primera persona como un empleado del banco." \
    # " Responde en un párrafo o menos." \
    prompt = \
        "Contexto: Eres experto en responder preguntas sobre el Banco de Chile." \
        " En el siguiente documento se encuentran los extractos que debes usar para responder." \
        " Siempre debes responder en primera persona como un empleado del banco." \
        f"\nDocumento: {dumps(matches.to_dict(), ensure_ascii=False)}" \
        f"\nPregunta: ¿Quién es el Gerente General de Banco de Chile?" \
        f"\nRespuesta: Eduardo Ebenperger Orrego." \
        f"\nPregunta: ¿Cuántos años tiene Banco de Chile operando en el país?" \
        f"\nRespuesta: 129 años." \
        f"\nPregunta: ¿Cuál es la misión del Banco de Chile?" \
        f"\nRespuesta: Contribuir al desarrollo del país y al progreso de las personas, mejorando su calidad de vida sin comprometer el bienestar de las futuras generaciones. Impulsamos el crecimiento económico de manera equilibrada con el medio ambiente y en igualdad de oportunidades." \
        f"\nPregunta: {question}" \
        f"\nRespuesta:"
    
    if print_prompt:
        print(prompt)
    
    unicorn_response = unicorn.predict(prompt)
    
    try:
        return unicorn_response.text
    except:
        return "Error!"

In [ ]:
ask_unicorn("¿Cuántas sucursales hay en Tarapacá?", print_prompt=False)

In [5]:
with open("./questions.txt", "r") as f:
    questions = f.read()
    questions = questions.split("\n")

with open("./questions_raw.txt", "r") as f:
    raws = f.read()
    raws = raws.split("\n")

csv_format = "{},\"{}\",\"{}\""

question_number = 1
for question, raw in zip(questions, raws):
    answer = ask_gemini(question, print_prompt=False)
    print(csv_format.format(question_number, raw, answer))
    question_number = question_number + 1

1,"¿Quién es el Gerente General de Banco de Chile?","Eduardo Ebenperger Orrego."
2,"¿Quién es el contacto de Investor Relations? ","Pablo Mejía Ricci."
3,"¿Cuántos años tiene Banco de Chile operando en el país?","129 años."
4,"¿Cuál es el Propósito de Banco de Chile?","El propósito de Banco de Chile es contribuir al desarrollo del país y las personas, cumpliendo de esta manera con cada uno de los compromisos asumidos con los distintos grupos de interés."
5,"¿Cuál es la Misión de Banco de Chile?","Somos una Corporación líder conectada al mundo, con una prestigiosa tradición de negocios."
6,"¿Cuál es la Visión de Banco de Chile?","Buscamos permanentemente, en todo lo que hacemos, ser el mejor banco para nuestros clientes, el mejor lugar para trabajar y la mejor inversión para nuestros accionistas."
7,"¿Cuáles son los Valores Corporativos de Banco de Chile?","Integridad, compromiso, respeto, lealtad, prudencia, responsabilidad y justicia."
8,"¿Cuántas sucursales tiene el Banco en el país?